In [41]:
import pandas as pd
import pendulum
from google.cloud import bigquery
from google.cloud.exceptions import NotFound
from unidecode import unidecode

In [ ]:
def updload_to_bq(
    df_: pd.DataFrame,
    project_id: str,
    dataset_id: str,
    table_id: str,
    add_load_dtttm: bool = False,
    partition_field: str = "fecha_carga",
    dataset_locatino: str = "us-east1",
) -> None:
    """Loads DataFrame to BigQuery.
    Skipped if DataFrame is empty.
    In case of error, it will retry with a suffix.

    Args:
        df_ (pd.DataFrame): DataFrame to upload, only uploaded if not empty.
        project_id (str): destination project id.
        dataset_id (str): destination dataset id.
        table_id (str): destination table id.
        add_load_dtttm (bool): define if add load_dttm column. Defaults to True.
        partition_field (str): name of the partition field. Only allows DAY partition.
            Defaults to "load_dttm".
    """
    if not df_.empty:
        client = bigquery.Client(project_id)

        try:
            client.get_dataset(dataset_id)  # Make an API request.
            print(f"Dataset {dataset_id} already exists")
        except NotFound:
            print(f"Dataset {dataset_id} not found, creating")
            dataset = bigquery.Dataset(project_id + "." + dataset_id)
            dataset.location = dataset_locatino
            client.create_dataset(dataset, timeout=30)
            print(f"Dataset {dataset_id} created")

        table = ".".join(
            [
                project_id,
                dataset_id,
                table_id,
            ]
        )

        print("Loading data to BigQuery")
        if add_load_dtttm:
            df_[partition_field] = pd.to_datetime(pendulum.now().isoformat())
        job_config = bigquery.LoadJobConfig()
        job_config.write_disposition = "WRITE_APPEND"
        job_config.schema_update_options = [
            bigquery.SchemaUpdateOption.ALLOW_FIELD_ADDITION
        ]
        job_config.time_partitioning = bigquery.TimePartitioning(
            type_=bigquery.TimePartitioningType.DAY, field=partition_field
        )
        try:
            load_job = client.load_table_from_dataframe(
                df_,
                table,
                job_config=job_config,
            )
            load_job.result()
            print("Dataframe loaded succesfully")
        except Exception as e:
            print(e)
            print("Cloud not load to specified table, retrying with suffix")
            now = pendulum.now("America/Santiago").format("YYYYMMDDHHmmss")
            load_job = client.load_table_from_dataframe(
                df_,
                table + now,
                job_config=job_config,
            )
            load_job.result()
            print(f"Dataframe loaded succesfully with the {now} suffix")
    else:
        print("No data to upload")

In [43]:
df_ = pd.read_excel("202105.xls").astype(str)

In [44]:
df_.columns = df_.columns.str.lower().str.replace(" ", "_").map(unidecode)

In [45]:
df_.insert(
    loc=0,
    column="fecha_carga",
    value=pd.to_datetime(pendulum.now("America/Santiago").isoformat()),
)

In [46]:
df_

,fecha_carga,id_cita,fecha,agenda,profesional,hora_inicio_cita,hora_fin_cita,tratamiento,codigo,rut,...,referencia,usuario_creacion,fecha_creacion_utc,prevision,online,simbolos,sucursal,venta_id_de_id_cita,ficha,origen
0,2025-06-06 17:15:30.434838-04:00,12748701,2021-05-28 00:00:00,Box 1 Quiropraxia,nan,15:00:00,15:20:00,nan,nan,nan,...,nan,Volans1,2021-05-26 17:19:57,nan,No,nan,Clínica Volans,nan,nan,Agenda
1,2025-06-06 17:15:30.434838-04:00,12748859,2021-05-28 00:00:00,Box 1 Quiropraxia,nan,14:50:00,15:10:00,nan,nan,nan,...,nan,Volans1,2021-05-26 17:24:11,nan,No,"Reconfirmado, no molestar más",Clínica Volans,nan,nan,Agenda
2,2025-06-06 17:15:30.434838-04:00,12748854,2021-05-28 00:00:00,Box 1 Quiropraxia,Dra. María Ignacia Pool C.,14:50:00,15:10:00,nan,nan,nan,...,Google Ads,Volans1,2021-05-26 17:24:02,Banmedica,No,"Reconfirmado, no molestar más",Clínica Volans,nan,20975899-8,Agenda
3,2025-06-06 17:15:30.434838-04:00,12749494,2021-05-27 00:00:00,Box 1 Quiropraxia,nan,09:20:00,09:40:00,nan,nan,nan,...,Google Ads,Volans1,2021-05-26 17:40:37,Banmedica,No,nan,Clínica Volans,nan,20975899-8,Agenda
4,2025-06-06 17:15:30.434838-04:00,12749494,2021-05-27 00:00:00,Box 1 Quiropraxia,nan,09:20:00,09:40:00,Blanqueamiento Dental En Casa (servicio clínico),nan,nan,...,Google Ads,Volans1,2021-05-26 17:40:37,Banmedica,No,nan,Clínica Volans,nan,20975899-8,Agenda
5,2025-06-06 17:15:30.434838-04:00,12748352,2021-05-25 00:00:00,Box 1 Quiropraxia,Dr. Andrés Contreras A.,09:30:00,10:00:00,nan,nan,nan,...,Google Ads,Volans1,2021-05-26 17:10:40,Banmedica,No,nan,Clínica Volans,nan,20975899-8,Agenda


In [47]:
updload_to_bq(
    df_=df_,
    project_id="clinica-volans",
    dataset_id="data_base",
    table_id="citas",
)

Dataset data_base already exists
Loading data to BigQuery


/home/andres/volans-parser/.venv/lib/python3.13/site-packages/google/cloud/bigquery/_pandas_helpers.py:489: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(


Dataframe loaded succesfully
